In [52]:
# in/out of the passage based dataset
dataset = "/home/thiziri/Documents/DOCTORAT/OSIRIM/projets/iris/PROJETS/WEIR/data/title_only/robust"  # passages_ranking_indri/Robust/MZ_data/concatenated/5"
# "/home/thiziri/Documents/DOCTORAT/SOFT/MatchZoo_latest1/data/AP88/passageRetrieval_qrels/concatenated"
output_index = "/home/thiziri/Documents/DOCTORAT/COLLECTION/Indri_index/Robust_title"  # Robust_passages"
# "/home/thiziri/Documents/DOCTORAT/OSIRIM/projets/iris/PROJETS/WEIR/collections/Indri_Index/passages_based/Robust"
# "/home/thiziri/Desktop/tst"
doc_templet = """<DOC>
    <DOCNO>doc_num</DOCNO>
    <TEXT>
    doc_text
    </TEXT>
</DOC>"""
indri = "/home/thiziri/Documents/DOCTORAT/SOFT/INDRI/indri-5.11"  # "/home/thiziri/Documents/DOCTORAT/OSIRIM/logiciels/indri-5.11/"
run_out = "/home/thiziri/Documents/DOCTORAT/OSIRIM/projets/iris/PROJETS/WEIR/RUNS/2ndYear/indri_runs/title_only"  # passages_based_robust"

In [53]:
# xml tools
from xml.etree import ElementTree
from xml.dom import minidom

def prettify(elem):
    """
    Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ").replace("<?xml version=\"1.0\" ?>\n", "")


"""
read unique values from column n in the file f
"""
def read_values(f, n):
    inf = open(f, "r")
    lines = inf.readlines()
    result = []
    for x in lines:
        result.append(x.split()[n])
    inf.close()
    return set(result)

In [54]:
# create dataset of the concatenated documents to index them with INDRI
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

for fold in tqdm(os.listdir(dataset)):
    # construct temporary corpus
    if not os.path.exists(os.path.join(output_index, fold)):
        os.mkdir(os.path.join(output_index, fold))
    queries = {}
    docs = []
    with open(os.path.join(os.path.join(output_index, fold), "corpus.txt"), 'w') as out_file:
        with open(os.path.join(os.path.join(dataset, fold), "corpus.txt"), 'r') as in_file:
            for line in in_file:
                tokens = line.strip().split()
                if tokens[0].isalnum():
                    queries[tokens[0]] = " ".join(tokens[1:])  # get queries texts
                else:
                    doc_id = tokens[0]  # .split('_')[0]  # we could have the same doc_id with diff contents for diff queries
                    docs.append(doc_templet.replace("doc_num", doc_id).replace("doc_text", " ".join(tokens[1:-1])))
        out_file.write('\n'.join(docs))  # write the documents mini-collection
        c_root = ET.Element("parameters")
        for q_id in queries:  # create the corresponding query-file for indri
            # print(q_id, q_txt)
            c_query = ET.SubElement(c_root, "query")
            type_ = ET.SubElement(c_query, "type")
            type_.text = "indri"
            num = ET.SubElement(c_query, "number")
            num.text = q_id
            text = ET.SubElement(c_query, "text")
            text.text = queries[q_id]
        with open(os.path.join(os.path.join(output_index, fold), "queries.xml"), 'w') as q_out:
            q_out.write(prettify(c_root))
    break  # only one fold, because file "corpus.txt" contains all queries and documents

  0%|          | 0/6 [00:00<?, ?it/s]

In [55]:
# index and query with INDRI:
for fold in tqdm(os.listdir(output_index)):
    if os.path.isdir(os.path.join(output_index, fold)):
        # create the index parameter file:
        with open(os.path.join(os.path.join(output_index, fold), "index_parameter.xml"), 'w') as q_out:
            c_root = ET.Element("parameters")
            mem = ET.SubElement(c_root, "memory")
            mem.text = "2000m"
            index = ET.SubElement(c_root, "index")
            index.text = os.path.join(os.path.join(output_index, fold), "index")
            stem = ET.SubElement(c_root, "stemmer")
            stem_name = ET.SubElement(stem, "name")
            stem_name.text = "krovetz"
            corp = ET.SubElement(c_root, "corpus")
            corp_path = ET.SubElement(corp, "path")
            corp_path.text = os.path.join(os.path.join(output_index, fold), "corpus.txt")
            corp_class = ET.SubElement(corp, "class")
            corp_class.text = "trectext"
            field = ET.SubElement(c_root, "field")
            name = ET.SubElement(corp, "name")
            name.text = "TEXT"
            q_out.write(prettify(c_root))
                
        # perform indexing AND ranking:
        os.chdir(indri)
        cmd = "buildindex/IndriBuildIndex " + os.path.join(os.path.join(output_index, fold), "index_parameter.xml")
        print("Indexing ...")
        os.system(cmd)
        
        cmd = "runquery/IndriRunQuery " + os.path.join(os.path.join(output_index, 
                                                                    fold), 
                                                       "queries.xml") + " -count=1000 -index=" + os.path.join(os.path.join(output_index, 
                                                                                                                           fold), 
                                                                                                              "index") + \
              " -trecFormat=true -baseline=okapi,k1:1.2,b:0.75,k3:7 > " + os.path.join(run_out, "okapi_k1_1.2_b_0.75_k3_7" + fold)
        print("Retrieving ...")
        os.system(cmd)
    

  0%|          | 0/1 [00:00<?, ?it/s]

Indexing ...


Retrieving ...


100%|██████████| 1/1 [00:12<00:00, 12.40s/it]

In [58]:
# aggregate the different passages of a same document: needed for documents represented by passages ONLY 

from collections import defaultdict
from numpy import average as avg
import operator

for file in tqdm(os.listdir(run_out)):
    uniques = defaultdict(dict)
    f = open(os.path.join(run_out, file), 'r')
    for line in f.readlines():
        if len(line.strip()) > 0:
            tokens = line.strip().split()
            doc_id, q_id = tokens[2].split('_')
            score = float(tokens[4])
            if doc_id not in uniques[q_id]:
                uniques[q_id][doc_id] = [score] 
            else:
                uniques[q_id][doc_id].append(score)
    # average scores:
    scores = defaultdict(dict)
    for q in tqdm(uniques):
        for doc_id in uniques[q]:
            scores[q][doc_id] = round(avg(uniques[q][doc_id]), 5)
    # write results:
    with open(os.path.join(run_out, file + "_aggreg"), 'w') as out:
        sorted_scores = scores  # dict(sorted(scores.items(), key=lambda kv: kv[1]))
        for q in tqdm(sorted_scores):
            sorted_docs = sorted(sorted_scores[q].items(), key=operator.itemgetter(1), reverse=True)
            rank = range(1, len(sorted_docs)+1)
            iter_rank = iter(rank)
            q_results = ["{q} Q0 {d} {i} {s} indri".format(q=q, d=e[0], i=next(iter_rank), 
                                                           s=e[1]) for e in sorted_docs]
            out.write('\n'.join(q_results) + '\n')

  0%|          | 0/1 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 2, got 1)